In [1]:
%%html
<style>@font-face{font-family:Vazir;src:url(Vazir.eot);src:url(Vazir.eot?#iefix) format("embedded-opentype"),url(Vazir.woff2) format("woff2"),url(Vazir.woff) format("woff"),url(Vazir.ttf) format("truetype");font-weight:400;font-style:normal}.farsi{font-family:Vazir;direction:rtl}.center{text-align:center}.justify{text-align:justify}.blue{color:blue}</style>

In [2]:
import pandas as pd
import numpy as np

CIFAR_LENGTH  = 32
train_data    = pd.read_csv('CIFAR10/CIFAR10_train_data.csv' ,header=None)
train_label   = pd.read_csv('CIFAR10/CIFAR10_train_label.csv',header=None)
test_data     = pd.read_csv('CIFAR10/CIFAR10_val_data.csv'   ,header=None)
test_label    = pd.read_csv('CIFAR10/CIFAR10_val_label.csv'  ,header=None)
real_test     = pd.read_csv('CIFAR10/CIFAR10_test_data.csv'  ,header=None)
np_test_label = np.array(test_label[0].values)
print('loading........done')

loading........done


In [3]:
import torch
calc_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(calc_device)

cuda


In [48]:
import torch
import torchvision
import torchvision.transforms as transforms

# classes = list(set(train_label[0].values))
transform = transforms.Compose([transforms.ToTensor()])

# trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
#                                         download=True, transform=transform)
# train_data['labels'] = train_label[0].apply(lambda x:classes.index(x))
# trainloader = torch.utils.data.DataLoader(train_data.values, batch_size=32)

# testset = torchvision.datasets.CIFAR10(root='./data', train=False,
#                                        download=True, transform=transform)
# testloader = torch.utils.data.DataLoader(testset, batch_size=4,
#                                          shuffle=False, num_workers=2)

bs = 32

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=bs,shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=bs,shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


In [49]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [50]:
import torch.optim as optim

net = Net().to(device = calc_device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(5):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels] 
        inputs, labels = data[0].to(calc_device), data[1].to(calc_device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        if i % 500 == 499:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 500))
            running_loss = 0.0

print('Finished Training')


[1,   500] loss: 2.302
[1,  1000] loss: 2.297
[1,  1500] loss: 2.263
[2,   500] loss: 2.166
[2,  1000] loss: 2.065
[2,  1500] loss: 1.929
[3,   500] loss: 1.816
[3,  1000] loss: 1.725
[3,  1500] loss: 1.658
[4,   500] loss: 1.602
[4,  1000] loss: 1.563
[4,  1500] loss: 1.540
[5,   500] loss: 1.505
[5,  1000] loss: 1.490
[5,  1500] loss: 1.486
Finished Training


In [51]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels =  data[0].to(calc_device), data[1].to(calc_device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 48 %
